In [2]:
%cd ..

/Users/mateoibarguen/Desktop/DATA-Capstone


In [3]:
from models.networks.lstm_models import *
from models.spectra_preprocessor import *

In [9]:
spectra_pp = SpectraPreprocessor(dataset_name='test', num_channels=5, num_instances=100, use_generator=False)

dataset_name:  test
subset_name:  train
dataset_name:  test
subset_name:  test


In [10]:
X_train, y_train, X_test, y_test = spectra_pp.transform(encoded=True)

In [12]:
a = np.array([[0, 1, 2], [3, 4, 5]])
np.pad(a, ((1, 0), (0, 0)), 'symmetric')

array([[0, 1, 2],
       [0, 1, 2],
       [3, 4, 5]])

In [13]:
class GoogleModel(BaseModel):

    def set_params_range(self):
        return {'conv_1': {'type': 'integer', 'min': 8, 'max': 64, 'default': 16},
                'conv_2': {'type': 'integer', 'min': 8, 'max': 64, 'default': 32},
                'bi_1': {'type': 'float', 'min': 8, 'max': 128, 'default': 128},
                'bi_2': {'type': 'integer', 'min': 8, 'max': 128, 'default': 128},
                'drop_1': {'type': 'float', 'min': 0, 'max': 1, 'default': 0.01},
                'dense_1': {'type': 'float', 'min': 8, 'max': 128, 'default': 64},
                'drop_2': {'type': 'float', 'min': 0, 'max': 1, 'default': 0.05}
                }

    def build_model(self, num_channels, num_timesteps, output_shape, params):
        """
        From: https://github.com/douglas125/SpeechCmdRecognition/blob/master/SpeechModels.py

        """
        model = Sequential()
        model.add(Conv1D(params['conv_1'], 5, input_shape=(num_timesteps, num_channels)))
        model.add(BatchNormalization())
        model.add(Conv1D(params['conv_2'], 5))
        model.add(BatchNormalization())
        model.add(Bidirectional(LSTM(params['bi_1'], return_sequences=True)))
        model.add(Bidirectional(LSTM(params['bi_2'], return_sequences=True)))
        model.add(Attention(293))
        model.add(Dropout(params['drop_1']))
        model.add(Dense(params['dense_1'], activation='elu'))
        model.add(Dropout(params['drop_2']))
        model.add(Dense(output_shape, activation='softmax'))
        return model

In [14]:
g = GoogleModel(num_channels=10, num_timesteps=301, output_shape=5)

In [16]:
#g.fit(X_train, y_train, X_test, y_test, 64, 1, compile_dict=baseline_model_compile_dict)